# Cats VS Dogs - Transfer Learning

In [1]:
# create directories
from os import makedirs
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/','test/']
for subdir in subdirs:
    labeldirs = ['dogs/','cats/']
    for labeldir in labeldirs:
        newdir = dataset_home + subdir + labeldir
        makedirs(newdir,exist_ok=True)

Now change your dataset to fit this format

In [2]:
from os import makedirs
from os import listdir
from shutil import copyfile

# create directories
from random import seed,random
seed(1)

# define ratio of pictures to use for validation
val_ratio = 0.25

# copy training dataset images into subdirectories
src_directory = r"C:\Users\mohit\Desktop\data science\Deep Leraning\Cats VS Dogs - Transfer Learning\dogs_vs_cats\train"
dataset_home = 'dataset_dogs_vs_cats/'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('cat'):
        dst = dataset_home + dst_dir + 'cats/' + file
        copyfile(src,dst)
    elif file.startswith('dog'):
        dst = dataset_home + dst_dir + 'dogs/' + file
        copyfile(src,dst)
        

Prepare a model for Transfer Learning

In [3]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
model = VGG16(include_top = False, input_shape = (224,224,3))
# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128,activation = 'relu',kernel_initializer = 'he_uniform')(flat1)
output = Dense(1,activation = 'sigmoid')(class1)
# define new model
model = Model(inputs = model.inputs, outputs = output)
# compile model
opt = SGD(learning_rate = 0.001, momentum = 0.9)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

Run the Model using Generator

In [ ]:
datagen = ImageDataGenerator(featurewise_center = True)
# specify imagenet mean values for centering
datagen.mean = [123.68,116.779,103.939]
# prepare iterator
train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',class_mode = 'binary',batch_size = 64,target_size = (224,224))
test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',class_mode = 'binary',batch_size = 64,target_size = (224,224))
# fit model
history = model.fit_generator(train_it,steps_per_epoch = len(train_it),validation_data = test_it,validation_steps= len(test_it),epochs = 5,
                             verbose = 1)

Plot the loss and accuracy

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'],color = 'blue',label = 'train')
plt.plot(history.history['val_loss'], color = 'orange',label = 'test')
# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'],color = 'blue',label = 'train')
plt.plot(history.history['val_accuracy'],color = 'orange',label = 'test')

Save the model for future use

In [ ]:
model.save('final_model.h5')